## Correlation matrices and eigenvalues of merged per-log dataframes:

### TODO: Still need to get energy numbers + eigenvalues of corr_matrix of per-us dataframes

In [29]:
import os
import pandas as pd
from numpy.linalg import eig, eigvalsh

import eigen_analysis

In [34]:
def get_latencies(out_fname):
    with open(out_fname, 'r') as f:
        lines = f.readlines()
    header = lines[0].rstrip('\n').split()
    read_lat = lines[1].rstrip('\n').split()
    lat = {'read': dict(zip(header[1:], [float(y) for y in read_lat[1:]]))}

    return lat['read']

In [38]:
corr_itr_list = {}

all_itr_dir = '200k_0xd00_merged_per_core_dfs/'
rapl = '135'
run = '0'
qps = all_itr_dir.split('_')[0]
dvfs = all_itr_dir.split('_')[1]

for file in os.listdir(all_itr_dir):
    itr = file.split('_')[2]
    print('ITR_DELAY: ', itr)
    df = pd.read_csv(all_itr_dir + file, sep = ',')
    df = df.drop(df.columns[0], axis=1)
    df_corr = df.corr()
    corr_itr_list[itr] = df_corr

ITR_DELAY:  2
ITR_DELAY:  20
ITR_DELAY:  300
ITR_DELAY:  200
ITR_DELAY:  250
ITR_DELAY:  40
ITR_DELAY:  30
ITR_DELAY:  400
ITR_DELAY:  50
ITR_DELAY:  10
ITR_DELAY:  350
ITR_DELAY:  100


In [70]:
eig_itr_list = {}

for itr in corr_itr_list.keys():
    df_corr = corr_itr_list[itr]
    vals, vecs = eig(df_corr)
    i = 0
    vals_dict = {}
    for val in vals:
        if val <= 0:
            print('EIGENVAL <= 0')
        vals_dict['eig_' + str(i)] = val
        i += 1
    eig_itr_list[itr] = vals_dict

In [72]:
#eig_itr_list

In [73]:
lat_itr_list = {}
lat_dir = '200k_qps/linux_mcd_out_0_0xd00_135_200k/'

for itr in corr_itr_list.keys():
    lat_file = lat_dir + 'linux.mcd.out.0_' + str(itr) + '_0xd00_135_200000'
    latencies = get_latencies(lat_file)
    lat_itr_list[itr] = latencies

In [74]:
#lat_itr_list

In [75]:
lat_columns = []
for itr in list(lat_itr_list.keys())[0]:
    lat_columns = list(lat_itr_list[itr].keys())
    
print(lat_columns)

['avg', 'std', 'min', '5th', '10th', '50th', '90th', '95th', '99th']


In [76]:
eig_columns = []
for itr in list(eig_itr_list.keys())[0]:
    eig_columns = list(eig_itr_list[itr].keys())

print(eig_columns)

['eig_0', 'eig_1', 'eig_2', 'eig_3', 'eig_4']


In [77]:
all_cols = lat_columns + eig_columns
all_cols.append('desc')
print(all_cols)

['avg', 'std', 'min', '5th', '10th', '50th', '90th', '95th', '99th', 'eig_0', 'eig_1', 'eig_2', 'eig_3', 'eig_4', 'desc']


In [80]:
eig_lat_energy_df = pd.DataFrame(columns=lat_columns + eig_columns)

for itr in corr_itr_list.keys():
    desc = run + '_' + itr + '_' + dvfs + '_' + rapl + '_' + qps
    new_row = {}
    for p in lat_columns:
        new_row[p] = lat_itr_list[itr][p]
    for e in eig_columns:
        new_row[e] = eig_itr_list[itr][e]
    new_row['desc'] = desc
    eig_lat_energy_df = eig_lat_energy_df._append(new_row, ignore_index = True)

In [81]:
eig_lat_energy_df

,avg,std,min,5th,10th,50th,90th,95th,99th,eig_0,eig_1,eig_2,eig_3,eig_4,desc
0,71.3,17.4,41.1,55.1,56.2,66.2,93.0,104.9,134.4,3.437880,0.957107,0.485085,0.119927,8.150684e-08,0_2_0xd00_135_200k
1,74.0,14.6,49.8,57.1,59.7,71.1,92.8,102.1,124.4,3.479148,0.930026,0.466559,0.124266,2.093437e-08,0_20_0xd00_135_200k
2,237.6,85.6,49.8,101.1,119.6,240.1,351.6,367.0,401.8,3.380390,0.991840,0.532754,0.094445,5.714871e-04,0_300_0xd00_135_200k
3,179.3,57.8,45.3,86.6,100.6,180.9,256.3,270.5,297.3,3.369317,0.974018,0.543048,0.112937,6.809913e-04,0_200_0xd00_135_200k
4,208.2,71.6,54.8,94.3,110.5,208.9,303.0,322.5,354.3,3.419246,0.984261,0.485820,0.110002,6.702868e-04,0_250_0xd00_135_200k
5,81.8,17.0,45.3,58.8,61.6,79.8,104.5,112.7,130.8,3.351228,0.944285,0.575131,0.129355,1.406585e-08,0_40_0xd00_135_200k
6,76.9,15.5,41.1,57.5,60.4,74.8,96.6,105.4,126.9,3.416435,0.961143,0.489520,0.132902,3.800894e-08,0_30_0xd00_135_200k
7,292.5,113.5,49.8,111.7,135.7,294.9,442.9,471.3,515.1,3.436523,0.992430,0.473820,0.096801,4.257090e-04,0_400_0xd00_135_200k
8,87.9,19.3,45.3,60.9,63.5,86.5,113.4,120.7,140.1,3.247200,0.975741,0.635845,0.141214,1.210419e-07,0_50_0xd00_135_200k
9,72.4,14.7,45.3,56.4,58.3,68.4,91.1,101.7,125.3,4.996039,0.002492,0.001076,0.000393,5.770076e-11,0_10_0xd00_135_200k


In [82]:
#corr_itr_list['2']

In [83]:
#corr_itr_list['200']

In [84]:
#corr_itr_list['250']

In [85]:
#corr_itr_list['300']

In [86]:
#corr_itr_list['30']

In [87]:
#corr_itr_list['400']